In [1]:
import sys
sys.path.insert(0, '../')
import gc
import logging
logging.basicConfig(level=logging.DEBUG)
from time import time
import os
import numpy as np
np.random.seed(42)

from cryptotrader.envs.driver import Apocalipse, get_historical
from cryptotrader.envs.utils import make_env, get_dfs_from_db
from cryptotrader.utils import convert_to, array_normalize
from cryptotrader.models import cn_models

import chainer as cn
import pandas as pd
from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline
# %load_ext line_profiler

Loading BokehJS ...

In [2]:
seed = 42
epilson = 1e-8
outdir = './logs'
data_dir = './data/'
save_dir = './save/'

# Env params
freq = 30
obs_steps = 300
tax = 0.0025
init_fiat = 100.0
init_crypto = 0.0
max_eval_steps = 500
eval_runs = 1500
test = True

In [3]:
def convert_and_clean(x):
    x = x.apply(convert_to.decimal)
    f = x.rolling(30, center=True, min_periods=1).mean().apply(convert_to.decimal)
    x = x.apply(lambda x: x if x.is_finite() else np.nan)
    return x.combine_first(f)    

In [4]:
# Setup train environment
train_env = Apocalipse(name='train_env', seed=seed)
# Set environment options
train_env.set_freq(freq)
train_env.set_obs_steps(obs_steps)

symbols = ['usdtbtc', 'usdteth', 'usdtetc', 'usdtxrp', 'usdtltc']
for i, symbol in enumerate(symbols):
    df = pd.read_csv(data_dir + symbol + '_fev_may.csv')
    df['date'] = df.date.apply(pd.to_datetime)
    df = df.set_index('date')
    for col in df.columns:
        df[col] = convert_and_clean(df[col])
    train_env.add_df(df=df, symbol=symbol)
    train_env.add_symbol(symbol)
    train_env.set_init_crypto(init_crypto, symbol)
    train_env.set_tax(tax, symbol)

gc.collect()
train_env.set_init_fiat(init_fiat)

# Clean pools
train_env._reset_status()
train_env.clear_dfs()
gc.collect()

train_env.set_training_stage(True)
train_env.set_observation_space()
train_env.set_action_space()
train_env.reset(reset_funds=True, reset_results=True, reset_global_step=True);

# Setup test environment
test_env = Apocalipse(name='test_env', seed=seed)
# Set environment options
test_env.set_freq(freq)
test_env.set_obs_steps(obs_steps)

symbols = ['usdtbtc', 'usdteth', 'usdtetc', 'usdtxrp', 'usdtltc']
for i, symbol in enumerate(symbols):
    df = pd.read_csv(data_dir + symbol + '_may.csv')
    df['date'] = df.date.apply(pd.to_datetime)
    df = df.set_index('date')
    for col in df.columns:
        df[col] = convert_and_clean(df[col])
    test_env.add_df(df=df, symbol=symbol)
    test_env.add_symbol(symbol)
    test_env.set_init_crypto(init_crypto, symbol)
    test_env.set_tax(tax, symbol)

gc.collect()
test_env.set_init_fiat(init_fiat)

# Clean pools
test_env._reset_status()
test_env.clear_dfs()
gc.collect()

test_env.set_training_stage(False)
test_env.set_observation_space()
test_env.set_action_space()
test_env.reset(reset_funds=True, reset_results=True, reset_global_step=True);

INFO:Cryptocoin arbiter agent logging file:[train_env Apocalipse initialization]
Apocalipse Initialized!
ONLINE MODE: False


INFO:Cryptocoin arbiter agent logging file:[<function Apocalipse.set_action_space at 0x7f591d574d90>]
Setting environment with 6 symbols.

INFO:Cryptocoin arbiter agent logging file:[test_env Apocalipse initialization]
Apocalipse Initialized!
ONLINE MODE: False


INFO:Cryptocoin arbiter agent logging file:[<function Apocalipse.set_action_space at 0x7f591d574d90>]
Setting environment with 6 symbols.



In [17]:
eiie = cn_models.EIIE(train_env.get_step_obs_all().shape, 16, 64)

In [18]:
optimizer = cn.optimizers.Adam(alpha=5e-6)
optimizer.setup(eiie)

In [ ]:
training_epochs = 1000000
test_interval = 333
test_epochs = 32
batch_size = 32

def make_batch(env, batch_size):
    obs_batch = []
    target_batch = []
    for i in range(batch_size):
        # Choose some random index
        env.step_idx = env.offset + np.random.randint(high=env.df.shape[0] - env.offset - 1, low=0)
        # Get obs and target and append it to their batches
        obs_batch.append(train_env.get_step_obs_all().values)

        target = (env.df.loc(axis=1)[:,'close'].iloc[env.step_idx + 1].values /
                  env.df.loc(axis=1)[:,'open'].iloc[env.step_idx + 1].values) - 1
        target_batch.append(target.reshape([1,-1]))

    obs_batch = cn_models.batch_states(obs_batch, np, cn_models.phi)
    target_batch = np.swapaxes(cn_models.batch_states(target_batch, np, cn_models.phi), 3, 2)

    return obs_batch, target_batch

## Training loop
t0 = 1e-8
best_score = -np.inf
for epoch in range(training_epochs):
    try:
        t1 = time()

        obs_batch, target_train = make_batch(train_env, batch_size)

        prediction_train = eiie(obs_batch)

        loss = cn.functions.mean_squared_error(prediction_train, target_train)

        eiie.cleargrads()
        loss.backward()

        optimizer.update()

        train_r2 = cn.functions.r2_score(prediction_train, target_train)

        t0 += time() - t1
        print("Training epoch %d/%d, loss: %f, r2: %f, samples/sec: %f" % (epoch + 1,
                                                                           training_epochs,
                                                                           loss.data,
                                                                           train_r2.data,
                                                                           (epoch + 1) * batch_size / t0), end='\r')
        
        if epoch % test_interval == 0 and epoch != 0:
            test_losses = []
            test_scores = []
            for _ in range(test_epochs):
                test_batch, target_test = make_batch(test_env, batch_size)

                # Forward the test data
                prediction_test = eiie(test_batch)

                # Calculate the loss
                loss_test = cn.functions.mean_squared_error(prediction_test, target_test)
                loss_test.to_cpu()
                test_losses.append(loss_test.data)

                # Calculate the accuracy
                test_r2 = cn.functions.r2_score(prediction_test, target_test)
                test_r2.to_cpu()
                test_scores.append(test_r2.data)
                
            if np.mean(test_scores) > best_score:
                best_score = np.mean(test_scores)
                print("\nNew best score:", best_score, end='\r')
                cn.serializers.save_npz(save_dir + 'eiie0.1.npz', eiie, compression=True)

            print('\nval loss:{:.04f} val r2 score:{:.04f}'.format(
                np.mean(test_losses), np.mean(test_scores)))
        
    except KeyboardInterrupt:
        print("\nInterrupted by the user. Best score:", best_score)
        break

Training epoch 334/1000000, loss: 0.019210, r2: -241.211823, samples/sec: 53.0919942
New best score: -2773.38
val loss:0.2442 val r2 score:-2773.3811
Training epoch 667/1000000, loss: 0.012222, r2: -114.135208, samples/sec: 53.1457432
New best score: -472.489
val loss:0.0572 val r2 score:-472.4894
Training epoch 1000/1000000, loss: 0.010046, r2: -218.536011, samples/sec: 53.226868
val loss:0.0745 val r2 score:-478.8584
Training epoch 1333/1000000, loss: 0.011498, r2: -312.958405, samples/sec: 53.217541
val loss:0.1001 val r2 score:-656.2310
Training epoch 1666/1000000, loss: 0.004838, r2: -88.201401, samples/sec: 53.2266486
val loss:0.1388 val r2 score:-634.1464
Training epoch 1999/1000000, loss: 0.011351, r2: -207.772263, samples/sec: 53.233024
val loss:0.0859 val r2 score:-783.8140
Training epoch 2332/1000000, loss: 0.007356, r2: -61.259613, samples/sec: 53.2518861
New best score: -228.31
val loss:0.0288 val r2 score:-228.3100
Training epoch 2665/1000000, loss: 0.004261, r2: -55.1432

In [15]:
act = cn_models.CashBias()

test_env._reset_status()
test_env.set_training_stage(False)
test_env.reset(reset_funds=True, reset_results=True, reset_global_step=True);

# Load best model
cn.serializers.load_npz(save_dir + 'eiie0.1.npz', obj=eiie)

In [ ]:
for i in range(train_env.df.shape[0] - obs_steps - 1):
    action = act(eiie(cn_models.batch_states([train_env.get_step_obs_all().values], np, cn_models.phi)))
    action = array_normalize(np.clip(action.data.flatten(), a_min=0.0, a_max=np.inf))
    train_env.step(action)
    print("Processing obs %d/%d" % (i + 1, train_env.df.shape[0] - obs_steps - 1), end='\r')
train_env.plot_results(train_env._get_results());

In [ ]:
for i in range(test_env.df.shape[0] - obs_steps - 1):
    action = act(eiie(cn_models.batch_states([test_env.get_step_obs_all().values], np, cn_models.phi)))
    action = array_normalize(np.clip(action.data.flatten(), a_min=0.0, a_max=np.inf))
    test_env.step(action)
    print("Processing obs %d/%d" % (i + 1, test_env.df.shape[0] - obs_steps - 1), end='\r')
test_env.plot_results(test_env._get_results());